In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import numpy as np
from urllib import parse
import requests
from bs4 import BeautifulSoup as bs
from utils import Utils

ut = Utils()

In [3]:
# 여행 카테고리 top10
res = requests.get('https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&sca=%EA%B5%AD%EB%82%B4%2F%ED%95%B4%EC%99%B8%2F%EC%97%AC%ED%96%89')
soup = bs(res.text, 'html.parser')

channel_list = soup.select('table .subject h1 a')[0:10]

channel_list = [x.text.strip() for x in channel_list]

In [4]:
channel_list

['빠니보틀 Pani Bottle',
 '곽튜브',
 'Rirang OnAir',
 '코이티비 KOITV',
 'EBS 컬렉션 - 라이프스타일',
 'Sophie Kim _하은',
 '진정부부',
 '끼룩푸드 seagull food',
 '원지의하루',
 '브레이너 제이의 숙면 여행']

In [5]:
# 유튜브 채널 정보 리스트로 만들기
subs, view, video, search = ut.extractYoutubeStatistics(soup.select('table .subject h3')[0:10])

print(subs)
print(view)
print(video)
print(search)

[190, 167, 131, 115, 94, 89, 86, 83, 78, 68]
[42581, 37746, 21038, 99519, 63221, 2969, 36418, 58230, 14472, 13483]
[301, 311, 270, 1882, 3700, 129, 353, 698, 351, 201]
[4904, 1817, 5352, 10887, 5579, 4934, 238, 659, 3147, 3929]


In [6]:
youtuber_rank = pd.DataFrame()

youtuber_rank['channel_name'] = channel_list
youtuber_rank['subscriber (10th)'] = subs
youtuber_rank['cum_view (10th)'] = view
youtuber_rank['video'] = video
youtuber_rank['search'] = search
youtuber_rank['category'] = '국내/해외/여행'

In [7]:
youtuber_rank.to_csv('./data/youtuber_rank_top10.csv', encoding='utf-8-sig', index=False)

In [8]:
# 유튜브 채널 정보 리스트로 만들기 (채널 직접 검색 시)
res = requests.get('https://youtube-rank.com/board/bbs/search.php?sfl=wr_subject%7C%7Cwr_content%7C%7Cdescription&stx=%EB%B0%B1%EC%A2%85%EC%9B%90')
soup = bs(res.text, 'html.parser')

channel_name, subs, view, video, category = ut.extractYoutubeStatistics2(soup)

baek = [channel_name, subs, view, video, 0, category]
baek

['백종원 PAIK JONG WON', 593, 98763, 560, 0, '음식/요리/레시피']

In [9]:
res = requests.get('https://youtube-rank.com/board/bbs/search.php?sfl=wr_subject%7C%7Cwr_content%7C%7Cdescription&stx=%ED%86%A0%EB%AA%A8%ED%86%A0%EB%AA%A8')
soup = bs(res.text, 'html.parser')

channel_name, subs, view, video, category = ut.extractYoutubeStatistics2(soup)

tomotomo = [channel_name, subs, view, video, 0, category]
tomotomo

['토모토모TomoTomo', 103, 23200, 206, 0, '미분류']

In [10]:
res = requests.get('https://youtube-rank.com/board/bbs/search.php?srows=10&gr_id=&sfl=wr_subject%7C%7Cwr_content%7C%7Cdescription%7C%7Cwr_link1&stx=%EC%97%AC%ED%96%89%EA%B0%80+%EC%A0%9C%EC%9D%B4&sop=and')
soup = bs(res.text, 'html.parser')

channel_name, subs, view, video, category = ut.extractYoutubeStatistics2(soup)

jay = [channel_name, subs, view, video, 0, category]
jay

['여행가 제이 [Jay world traveler]', 60, 14500, 380, 0, '국내/해외/여행']

In [11]:
additional = pd.DataFrame([baek, tomotomo, jay], 
             columns = ['channel_name', 'subscriber (10th)', 'cum_view (10th)', 'video', 'search', 'category'])

used_youtuber = youtuber_rank.append(additional, ignore_index=True)

C:\Users\TT21\AppData\Local\Temp\ipykernel_9408\2991888735.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  used_youtuber = youtuber_rank.append(additional, ignore_index=True)


In [12]:
used_youtuber

,channel_name,subscriber (10th),cum_view (10th),video,search,category
0,빠니보틀 Pani Bottle,190,42581,301,4904,국내/해외/여행
1,곽튜브,167,37746,311,1817,국내/해외/여행
2,Rirang OnAir,131,21038,270,5352,국내/해외/여행
3,코이티비 KOITV,115,99519,1882,10887,국내/해외/여행
4,EBS 컬렉션 - 라이프스타일,94,63221,3700,5579,국내/해외/여행
5,Sophie Kim _하은,89,2969,129,4934,국내/해외/여행
6,진정부부,86,36418,353,238,국내/해외/여행
7,끼룩푸드 seagull food,83,58230,698,659,국내/해외/여행
8,원지의하루,78,14472,351,3147,국내/해외/여행
9,브레이너 제이의 숙면 여행,68,13483,201,3929,국내/해외/여행


In [13]:
used_youtuber = used_youtuber.drop([2, 4, 5, 6, 7, 8, 9])

In [14]:
used_youtuber = used_youtuber.sort_values('subscriber (10th)', ascending=False)

In [15]:
used_youtuber

,channel_name,subscriber (10th),cum_view (10th),video,search,category
10,백종원 PAIK JONG WON,593,98763,560,0,음식/요리/레시피
0,빠니보틀 Pani Bottle,190,42581,301,4904,국내/해외/여행
1,곽튜브,167,37746,311,1817,국내/해외/여행
3,코이티비 KOITV,115,99519,1882,10887,국내/해외/여행
11,토모토모TomoTomo,103,23200,206,0,미분류
12,여행가 제이 [Jay world traveler],60,14500,380,0,국내/해외/여행


In [16]:
used_youtuber.to_csv('./data/used_youtuber.csv', encoding='utf-8-sig', index=False)